# Segmentation


## Import 

In [ ]:
import numpy as np
import random
from scipy.signal import convolve2d
import cv2 
from skimage.segmentation import chan_vese


## k-means algorithm via lloyd's algorithm
* For grey images
* based on lloyds algorithm from slides
* the feature used must be pixel intensity

In [2]:
def Kmeans_lloyd(k, image, random_seed, patience = 2):

    random.seed(random_seed) # set random seed

    # ectract intensity values, the relative position of the pixels does not matter
    shape = image.shape
    pixels = image.flatten().reshape(-1,1) # 1d array of pixels, shape (N, 1)
    centroids = np.random.uniform(pixels.min(), pixels.max(), size = k).reshape(1, -1) # shape (1, N)


    change_count = 0 # part of the stopping condition

    while True:

        # compute distance for all points to all centroids
        distances = np.abs(pixels - centroids) # (N,1) x (1, k) = (N , k)

        #put each pixel in the cluster belonging to the nearest centroid
        clusters = np.argmin(distances, axis= 1)

        # recompute centroids (position)
        new_centroids = np.zeros((1,k)) # empty numpy array
        
        for i in range(k):
                cluster_pixels = pixels[clusters == i]
                new_centroids[0, i] = cluster_pixels.mean()
                    
        

        # STOPPING CONDITION
        if np.allclose(new_centroids, centroids): # checks if the values have moved within a small margin. It returns True if the centroids have stopped moving
            change_count += 1
        else:
            change_count =0

        if change_count >= patience:
            break

        centroids = new_centroids
    
    # save points to points to clusters
    clusters = clusters.reshape(shape)

    return clusters, centroids

def kmeans_segment(k , image, random_seed, patience = 2):
    clusters, centroids = Kmeans_lloyd(k , image, random_seed, patience)

    centroids_flat = centroids.flatten()

    
    segmented_image = centroids_flat[clusters]
    return segmented_image



def kmeans_segment_binary(k , image, random_seed, patience = 2):
    
    clusters, _ = Kmeans_lloyd(k , image, random_seed, patience)

    binary_image = clusters.astype(np.uint8)

    return binary_image

# Otsu thresholding algorithm
* Based on the 1979_otsu_IEEESys.pdf document and the slides segmentation.pdf

It was pretty hard to understand the algorithm, so i have written which equations i used from the 1979_otsu_IEEESys paper, when relevant.


In [3]:
def otsu(image):

    # Compute histogram
    histogram, bin_edges = np.histogram(image, bins=256, range=(0, 256))

    # Normalize historgram and regard it as a probability distribution P(i) = h(i) /N
    p_i = histogram / np.sum(histogram) 

    total_mean = np.sum(np.arange(256) * p_i)  # mean intensity of the image

    max_variance = 0 # initial maximum variance
    threshold = 0   # initial threshold
    w0 = 0  # initial  probability for class 0
    w1 = 0  # initial probability for class 1 
    mu_k = 0.0
    epsilon = 1e-10
    L = len(p_i)  # number of bins = 256

    for i in range(L):
        w0 += p_i[i]  #  probability for class 0 cumulative (equation 2)
        w1 = 1 - w0  #  probability for class 1 (equation 3)

        if w0 < epsilon or w1 < epsilon: # makes sure we dont divide by zero:)
            continue

        mu_k += i * p_i[i]  # cumulative mean up to bin i

        mu0 =  mu_k / w0  # mean for class 0 (equation 4)

        mu1 = (total_mean - mu_k) / w1  # mean for class 1 (equation 5) 

        # Between class variance
        variance = w0 * w1 * (mu1 - mu0) ** 2  #(equation 14)

        if variance > max_variance: # find the maximum variance and corresponding threshold
            max_variance = variance
            threshold = i

    return threshold


#wrapper
def otsu_segment(image):
    threshold = otsu(image)
    segmented_image = (image >= threshold).astype(np.uint8)
    return segmented_image

## cleaning/denoising algorithm

after reading the description i realised that you could basically just perform a convolution operation using a kernel to sum, and then choosing the class of the given pixel by using the threshold

In [4]:
def denoising_algorithm(image, relative_threshold, iterations = 1, BigNeighborhood = True):

  neigborhood8 = np.array([
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 1]
    ])

  neigborhood4 = np.array([
        [0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]
    ])
  
  kernel = neigborhood8 if BigNeighborhood else neigborhood4 # choosing which neighborhood to use
  max_votes = kernel.sum()

  current_image = image.copy()

  for _ in range(iterations):
   
    votes = convolve2d(current_image, kernel, mode='same', boundary='fill', fillvalue=0)  # returns the votes for each pixel

    relative_votes = votes / max_votes

    current_image = (relative_votes >= relative_threshold).astype(np.uint8) #update image based on voting threshold

  return current_image
    


## Import and Feature extraction function

In [5]:
def load_image_greyscale(path):
    image = cv2.imread(path)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

def display_segmented_image(image):
    cv2.imshow('image',image.astype(np.uint8) * 255)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Import images

In [6]:
#required images
camera = load_image_greyscale("images/camera.png")
coins = load_image_greyscale("images/coins.png")
page = load_image_greyscale("images/page.png")
rocksample = load_image_greyscale("images/rocksample.png")

#optional image
coronary = load_image_greyscale("images/coronary.jpg")
#taken from https://tedrogersresearch.ca/2021/11/coronary-angiography-a-tool-to-diagnose-cad-in-acute-heart-failure/



## Perform segmentation and display segmentation: kmeans and Otsu
* Create and run your implementations of k-means and Otsu on Absalon’s images and
one or more of your choice.
* Comment on their similarity, dissimilarities?

In [7]:
#Segment images
#experiment variables:
k = 2
ex_random_seed = 41
ex_patience = 2


#camera
camera_segmented_kmeans = kmeans_segment_binary(k ,camera, ex_random_seed, ex_patience)
camera_segmented_otsu = otsu_segment(image= camera)

#coins
coins_segmented_kmeans = kmeans_segment_binary(k , coins, ex_random_seed, ex_patience)
coins_segmented_otsu = otsu_segment(image= coins)



# page
page_segmented_kmeans = kmeans_segment_binary(k , page, ex_random_seed, ex_patience)
page_segmented_otsu = otsu_segment(image= page)


#rocksample
rocksample_segmented_kmeans = kmeans_segment_binary(k ,rocksample, ex_random_seed, ex_patience)
rocksample_segmented_otsu = otsu_segment(image= rocksample)


#coronary
coronary_segmented_kmeans = kmeans_segment_binary(k ,coronary, ex_random_seed, ex_patience)
coronary_segmented_otsu = otsu_segment(image= coronary)

#display images

#camera
display_segmented_image(camera_segmented_kmeans)
display_segmented_image(camera_segmented_otsu)

#coins
display_segmented_image(coins_segmented_kmeans)
display_segmented_image(coins_segmented_otsu)


#page
display_segmented_image(page_segmented_kmeans)
display_segmented_image(page_segmented_otsu)


#rocksample
display_segmented_image(rocksample_segmented_kmeans)
display_segmented_image(rocksample_segmented_otsu)


#coronary
display_segmented_image(coronary_segmented_kmeans)
display_segmented_image(coronary_segmented_otsu)




## Perform denoising on segmented images
* Run several iterations of the segmentation denoising algorithm with varying voting
thresholds values. Comment on the results.

In [8]:
## Perform denoising
# experiment variable
ex_vote_threshold = 0.75
ex_iterations = 2
ex_BigNeighborhood = True

#camera
camera_denoised_kmeans = denoising_algorithm(camera_segmented_kmeans, ex_vote_threshold, ex_iterations 
, ex_BigNeighborhood)
camera_denoised_otsu = denoising_algorithm(camera_segmented_otsu, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)

#coins
coins_denoised_kmeans = denoising_algorithm(coins_segmented_kmeans, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)
coins_denoised_otsu = denoising_algorithm(coins_segmented_otsu, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)

#page
page_denoised_kmeans = denoising_algorithm(page_segmented_kmeans, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)
page_denoised_otsu = denoising_algorithm(page_segmented_otsu, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)

#rocksample
rocksample_denoised_kmeans = denoising_algorithm(rocksample_segmented_kmeans, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)
rocksample_denoised_otsu = denoising_algorithm(rocksample_segmented_otsu, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)

#coronary
coronary_denoised_kmeans = denoising_algorithm(coronary_segmented_kmeans, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)
coronary_denoised_otsu = denoising_algorithm(coronary_segmented_otsu, ex_vote_threshold, ex_iterations
, ex_BigNeighborhood)


#display images

#camera
display_segmented_image(camera_denoised_kmeans)
display_segmented_image(camera_denoised_otsu)

#coins
display_segmented_image(coins_segmented_kmeans)
display_segmented_image(coins_segmented_otsu)


#page
display_segmented_image(page_denoised_kmeans)
display_segmented_image(page_denoised_otsu)


#rocksample
display_segmented_image(rocksample_denoised_kmeans)
display_segmented_image(rocksample_denoised_otsu)


#coronary
display_segmented_image(coronary_denoised_kmeans)
display_segmented_image(coronary_denoised_otsu)




# kmeans on page image for k>2
* Run a k-means segmentation with k > 2. Can it help with the page image?

In [9]:
#page k =3
page_segmented_kmeans3 = kmeans_segment(k =3 ,image = page, random_seed =42, patience = 2)
display_segmented_image(page_segmented_kmeans3)

#page k =4
page_segmented_kmeans4 = kmeans_segment(k =4 ,image = page, random_seed =42, patience = 2)
display_segmented_image(page_segmented_kmeans4)

#page k =5
page_segmented_kmeans5 = kmeans_segment(k =5 ,image = page, random_seed =42, patience = 2)
display_segmented_image(page_segmented_kmeans5)

#page k =8
page_segmented_kmeans8 = kmeans_segment(k =8 ,image = page, random_seed =42, patience = 2)
display_segmented_image(page_segmented_kmeans8)

#page k =10
page_segmented_kmeans10 = kmeans_segment(k =10 ,image = page, random_seed =42, patience = 2)
display_segmented_image(page_segmented_kmeans10)

## Extra optional k-means experiment for coronary and rock-sample (Not used)





In [16]:
#page k =3
coronary_segmented_kmeans3 = kmeans_segment(k =3 ,image = coronary, random_seed =42, patience = 2)
display_segmented_image(coronary_segmented_kmeans3)


coronary_segmented_kmeans6 = kmeans_segment(k =6 ,image = coronary, random_seed =42, patience = 2)
display_segmented_image(coronary_segmented_kmeans6)

# testing scikit-learn segmentation algorithms on the page image
* You may want to try some of the algorithms available in scikit-learn, such as the
Chan-Vese segmentation.

In [11]:
## chan vase segmentation
#experiment variable
ex_lambda1 = 1
ex_lambda2 = 1
ex_tol = 1e-3
ex_max_num_iter = 200
ex_extended_output = False

# #camera
camera_chan_vese_segmentation = chan_vese(image = camera,mu= 0.1, lambda1= ex_lambda1, lambda2=ex_lambda2, tol= ex_tol, max_num_iter= ex_max_num_iter, extended_output= ex_extended_output)
display_segmented_image(camera_chan_vese_segmentation)


#coins
coins_chan_vese_segmentation = chan_vese(image = coins,mu= 0.01, lambda1= ex_lambda1, lambda2=ex_lambda2, tol= ex_tol, max_num_iter= ex_max_num_iter, extended_output= ex_extended_output)
display_segmented_image(coins_chan_vese_segmentation)

# #page
page_chan_vese_segmentation = chan_vese(image = page,mu= 0.01, lambda1= ex_lambda1, lambda2=ex_lambda2, tol= ex_tol, max_num_iter= ex_max_num_iter, extended_output= ex_extended_output)
display_segmented_image(page_chan_vese_segmentation)

# #rock sample
rocksample_chan_vese_segmentation = chan_vese(image = rocksample,mu= 0.005, lambda1= ex_lambda1, lambda2=ex_lambda2, tol= ex_tol, max_num_iter= ex_max_num_iter, extended_output= ex_extended_output)
display_segmented_image(rocksample_chan_vese_segmentation)

# #coronary
coronary_chan_vese_segmentation = chan_vese(image = coronary,mu= 0.01, lambda1= ex_lambda1, lambda2=1, tol= ex_tol, max_num_iter= ex_max_num_iter, extended_output= ex_extended_output)
display_segmented_image(coronary_chan_vese_segmentation)


